# Hyperparameter Tuning: MinNDAE
In this *Jupyter Notebook* the goal is to find the *optimal hyperparameters* for the `MinNDAE` model using the Kera's `MNIST` dataset as the baseline/standard dataset.

## Setup
Need to get the necessary packages ...

In [ ]:
# check for colab
if "google.colab" in str(get_ipython()):
  # install colab dependencies
  !pip install git+https://github.com/DiogenesAnalytics/autoencoder

## Get MNIST Data
Wille use `keras.datasets` to get the `MNIST` dataset, and then do some *normalizing* and *reshaping* to prepare it for use in training the *autoencoder*.

In [ ]:
# get necessary libs for data/preprocessing
import tensorflow as tf
from keras.datasets import mnist

# load the data
(x_train, _), (x_test, _) = mnist.load_data()

# preprocess the data (normalize)
x_train = x_train.astype("float32") / 255.
x_test = x_test.astype("float32") / 255.

# add grayscale dimension
x_train = tf.expand_dims(x_train, axis=-1)
x_test = tf.expand_dims(x_test, axis=-1)

# convert to tf datasets
train_ds = tf.data.Dataset.from_tensor_slices((x_train, x_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, x_test))

# set a few params
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

# update with batch/buffer size
train_ds = train_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

## Building Hypermodel
Here we need to define the *function* that will be used to build the *hyper model* for the `MinNDAE` class.

In [ ]:
from autoencoder.model.minimal import MinNDParams, MinNDAE
from autoencoder.training import build_encode_dim_loss_function

# set regularization factor
REG_FACTOR = 1.0 / (28.0 * 28.0)

# define the autoencoder model
def build_autoencoder(hp):
    # get encoding dimension
    encode_dim = hp.Int("encode_dim", min_value=1, max_value=(28 * 28), step=1)
    
    # get layer configs
    config = MinNDParams(
        l0={"input_shape": (28, 28, 1)},
        l2={"units": encode_dim},
        l3={"units": 28 * 28 * 1},
        l4={"target_shape": (28, 28, 1)},
    )

    # create model
    autoencoder = MinNDAE(config)
    
    # get custom loss func using decorator factory
    custom_loss = build_encode_dim_loss_function(encode_dim, regularization_factor=REG_FACTOR)()
    
    # select loss function
    autoencoder.compile(optimizer="adam", loss=custom_loss)

    # now return keras model
    return autoencoder.model

## Hyperparameter Search
Now we can begin the *hyperparameter search algorithm*.

In [ ]:
# get hyperparam tools
from keras.callbacks import EarlyStopping
from keras_tuner import RandomSearch

# setup tuner
tuner = RandomSearch(
    build_autoencoder,
    objective="val_loss",
    max_trials=100,
    directory="autoencoder_tuning/minndae",
    project_name=f"random_search_encode_dim_{REG_FACTOR}_reg",
    seed=42,
)

# create early stop call backs
stop_early = EarlyStopping(monitor="val_loss", patience=2)

# generate random search space for hyperparameters
tuner.search_space_summary()

# run the hyperparameter search
tuner.search(train_ds, epochs=3, validation_data=test_ds, callbacks=[stop_early])

In [ ]:
# get hyperparams of best model
best_hp = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
print("Best Hyperparameters:", best_hp)

In [ ]:
# get plotting libs
import matplotlib.pyplot as plt

# extract score/encode_dims from each trial
scores, encoding_dims = zip(
    *((trial.score, trial.hyperparameters["encode_dim"]) for trial in tuner.oracle.trials.values())
)

# Plotting a line chart
plt.scatter(encoding_dims, scores)
plt.title(f"Performance vs Encoding Dimension:\n{MinNDAE.__name__} / MNIST / {REG_FACTOR:0.4f} Regularization")
plt.axvline(x=best_hp["encode_dim"], color="r", linestyle="dashed", linewidth=2, label="optimal_encode_dim")
plt.axvline(x=32, color="y", linestyle="dashed", linewidth=2, label="keras_default")
plt.xlabel("Encoding Dimension")
plt.ylabel("Loss Metric")
plt.legend()
plt.show()